In [ ]:
!pip install statsmodels==0.12.1

In [ ]:
!pip install scikit_posthocs

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

import tensorflow as tf
import keras
from keras.layers import Flatten, Dense
from keras.layers.convolutional import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D

import warnings
import copy
from operator import itemgetter

from scipy.stats import friedmanchisquare
from scikit_posthocs import posthoc_nemenyi, posthoc_nemenyi_friedman, posthoc_conover_friedman

In [ ]:
# Check whether GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load Data

> GDP growth rate, quarterly 1989 Q1 -- 2021 Q3

> CPIH, quarterly 1989 Q1 -- 2021 Q3

> Unemploment rate, quarterly 1989 Q1 -- 2021 Q3

In [ ]:
def read_file(file_name):
  """
  This function takes in time series' file name and return the time series.
  :param file_name: the name of the file contains time series.
  :return: Series data.
  """
  data = pd.read_csv(file_name)
  df = pd.DataFrame(data)
  df['Date'] = pd.to_datetime(df['Time'].str.replace(r'(\d+) (Q\d)', r'\1-\2'), errors='coerce')
  data = df.drop(columns=['Time'],axis=1).set_index('Date')
  return data

In [ ]:
warnings.filterwarnings("ignore")
gdp = read_file('UK_GDP.csv')
inflation = read_file('UK_inflation.csv')
unemployment = read_file('UK_unemployment.csv')

# ADF test

In [ ]:
def adf_test(data):
  """
  This function takes in time series and check for stationarity.
  :param data: a time series.
  :return: Float p-value.
  """ 
  data = data.squeeze().dropna() 
  test = adfuller(data)
  output = pd.Series(test[0:2], index=['Test Statistic', 'p-value'])
  for key, value in test[4].items():
    output['Critical Value %s' % key] = value
  return test[1]

In [ ]:
adf_result = {}

gdp_adf = adf_test(gdp)
adf_result["GDP"] = gdp_adf
if gdp_adf > 0.05:
  gdp_diff1 = gdp.diff()
  gdp_diff1_adf = adf_test(gdp_diff1)
  adf_result["GDP after first difference"] = gdp_diff1_adf

inflation_adf = adf_test(inflation)
adf_result["Inflation"] = inflation_adf
if inflation_adf > 0.05:
  inflation_diff1 = inflation.diff()
  inflation_diff1_adf = adf_test(inflation_diff1)
  adf_result["Inflation after first difference"] = inflation_diff1_adf

unemployment_adf = adf_test(unemployment)
adf_result["Unemployment"] = unemployment_adf
if unemployment_adf > 0.05:
  unemployment_diff1 = unemployment.diff()
  unemployment_diff1_adf = adf_test(unemployment_diff1)
  adf_result["Unemployment after first difference"] = unemployment_diff1_adf

adf_result

# Data Normalization

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

gdp_scaler = scaler.fit_transform(gdp)
gdp_scaler = pd.DataFrame(data=gdp_scaler, index=gdp.index)

inflation_scaler = scaler.fit_transform(inflation)
inflation_scaler = pd.DataFrame(data=inflation_scaler, index=inflation.index)

unemployment_scaler = scaler.fit_transform(unemployment)
unemployment_scaler = pd.DataFrame(data=unemployment_scaler, index=unemployment.index)

# Data Visualization

In [ ]:
plt.figure(figsize=(20,12))
plt.subplot(331)
plt.plot(gdp)
plt.subplot(332)
plt.plot(gdp_diff1)
plt.subplot(333)
plt.plot(gdp_scaler)

In [ ]:
plt.figure(figsize=(20,12))
plt.subplot(221)
plt.plot(inflation)
plt.subplot(222)
plt.plot(inflation_scaler)


In [ ]:
plt.figure(figsize=(20,12))
plt.subplot(331)
plt.plot(unemployment)
plt.subplot(332)
plt.plot(unemployment_diff1)
plt.subplot(333)
plt.plot(unemployment_scaler)

# Seperate data



> Train - Validate - Test

> Train - Test





In [ ]:
def train_validate_test_split(data):
  """
  This function takes in time series and split to train, validate and test sets.
  :param data: a time series.
  :return: Series train, validate and test.
  """
  train_size = int(len(data) * train_decimal)
  validate_size = int(len(data) * validate_decimal)
  
  train = data[:train_size]
  validate = data[train_size:train_size+validate_size]
  test = data[train_size+validate_size:]
  return train, validate, test

In [ ]:
def train_test_split(data):
  """
  This function takes in time series and split to train and test sets.
  :param data: a time series.
  :return: Series train and test.
  """
  new_train_size = int(len(data) * new_train_decimal)
  new_train = data[:new_train_size]
  new_test = data[new_train_size:]
  return new_train, new_test

In [ ]:
np.random.seed(9)
train_decimal = 0.6
validate_decimal = 0.2
new_train_decimal = 0.8

gdp_train_stg1, gdp_validate_stg1, gdp_test_stg1 = train_validate_test_split(gdp_scaler)
inflation_train_stg1, inflation_validate_stg1, inflation_test_stg1 = train_validate_test_split(inflation_scaler)
unemployment_train_stg1, unemployment_validate_stg1, unemployment_test_stg1 = train_validate_test_split(unemployment_scaler)

gdp_train_stg2, gdp_test_stg2 = train_test_split(gdp_scaler)
inflation_train_stg2, inflation_test_stg2 = train_test_split(inflation_scaler)
unemployment_train_stg2, unemployment_test_stg2 = train_test_split(unemployment_scaler)

# Algorithms

## ARIMA

In [ ]:
def d_value_choose(data):
  """
  This function takes in time series and decide the d value use in ARIMA model.
  :param data: a time series.
  :return: Integer d value.
  """
  d_value = np.inf
  for i in range(3):
    stationary = adf_test(data)
    if stationary <= 0.05:
      d_value = i
      break
    else:
      data = data.diff()
  return d_value

In [ ]:
def arima(train, validate, parameters):
  """
  This function takes in train and validate data as well as parameters and calculate 
  the RMSE between the predictions and reals values from validate set.
  :param train: train set of time series
  :param validate: validate set of time series
  :param parameters: parameters used as the configuration of model.
  :return: Float RMSE value.
  """
  num_logs = parameters[1]
  history = []
  for x in train.values:
    history.append(x)
  predictions = []
  for t in range(len(validate)):
    model = ARIMA(history, order=(num_logs, d_value_choose(train), 0)).fit()
    predict = model.forecast()[0]
    predictions.append(predict)
    valid = validate.values[t]
    history.append(valid)
  rmse = np.sqrt(mean_squared_error(validate.values, predictions))
  predictions = pd.Series(predictions, index=validate.index)
  return rmse

## Covolutional neural network

In [ ]:
def data_num_logs_split(data, num_logs):
  """
  This function takes in time series data and splits it into x and y values.
  :param data:time series dataset
  :param num_logs: num_logs is how many values in the time series you need for predicting the future values.
  :return: Tuple containing an Array of X values and an Array of y values.
  """
  X, y = [], []
  X_date, y_date = [], []
  for i in range(len(data)):
    end_index = i + num_logs
    if end_index > len(data) - 1:
      break
    log_xs, log_y = data.values.squeeze()[i:end_index], data.values.squeeze()[end_index]
    date_xs, date_ys = data.index[i:end_index], data.index[end_index]
    X.append(log_xs)
    y.append(log_y)
    X_date.append(date_xs)
    y_date.append(date_ys)
  return np.array(X), np.array(y), X_date, y_date

### CNN univariate:

 

> For single time series input.




In [ ]:
def cnn_univariate_stg1(train, validate, test, parameters):
  """
  This function takes in train and validate data as well as parameters to calculate 
  the RMSE between the predictions and reals values from validate set.
  :param train: train set of time series.
  :param validate: validate set of time series.
  :param test: test set of time series.
  :param parameters: parameters used as the configuration of model.
  :return: Float RMSE value on validate set and RMSE value on test set.
  """
  num_epochs, num_logs, num_hidden_layers, num_hidden_nodes, activation, optimizer = parameters[0], \
  parameters[1], parameters[2], parameters[3], parameters[4], parameters[5]
  X_train, y_train, X_train_date, y_train_date = data_num_logs_split(train, num_logs)
  X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
  X_validate, y_validate, X_validate_date, y_validate_date = data_num_logs_split(validate, num_logs)
  X_validate = X_validate.reshape((X_validate.shape[0], X_validate.shape[1], 1))
  X_test, y_test, X_test_date, y_test_date = data_num_logs_split(test, num_logs)
  X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

  cnn_single_model = keras.Sequential()
  cnn_single_model.add(Conv1D(filters=64, kernel_size=2, activation=activation, input_shape=(num_logs, 1)))
  cnn_single_model.add(MaxPooling1D())
  cnn_single_model.add(Flatten())
  for layer_index in range(num_hidden_layers):
    cnn_single_model.add(Dense(num_hidden_nodes, input_dim=num_logs, activation=activation))
  cnn_single_model.add(Dense(1))
  cnn_single_model.compile(optimizer=optimizer, loss='mse')

  cnn_single_model.fit(X_train, y_train, batch_size=100, epochs=num_epochs, verbose=2)
  
  y_validate_predict = []
  for X_validate_fragment in X_validate:
    X_validate_fragment = np.array(X_validate_fragment)
    X_validate_fragment = X_validate_fragment.reshape((1, num_logs, 1))
    y_predict = cnn_single_model.predict(X_validate_fragment, verbose=0)
    y_validate_predict.append(y_predict[0][0])
  rmse = np.sqrt(mean_squared_error(y_validate, y_validate_predict))

  y_test_predict = []
  for X_test_fragment in X_test:
    X_test_fragment = np.array(X_test_fragment)
    X_test_fragment = X_test_fragment.reshape((1, num_logs, 1))
    y_predict = cnn_single_model.predict(X_test_fragment, verbose=0)
    y_test_predict.append(y_predict[0][0])
  test_rmse = np.sqrt(mean_squared_error(y_test, y_test_predict))

  return rmse, test_rmse

In [ ]:
def cnn_univariate_stg2(train, test, parameters):
  """
  This function takes in train and test data as well as parameters to calculate 
  the RMSE between the predictions and reals values from test set.
  :param train: train set of time series.
  :param test: test set of time series.
  :param parameters: parameters used as the configuration of model.
  :return: Float RMSE value on test set.
  """
  num_epochs, num_logs, num_hidden_layers, num_hidden_nodes, activation, optimizer = parameters[0], \
  parameters[1], parameters[2], parameters[3], parameters[4], parameters[5]
  X_train, y_train, X_train_date, y_train_date = data_num_logs_split(train, num_logs)
  X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
  X_test, y_test, X_test_date, y_test_date = data_num_logs_split(test, num_logs)
  X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

  cnn_single_model = keras.Sequential()
  cnn_single_model.add(Conv1D(filters=64, kernel_size=2, activation=activation, input_shape=(num_logs, 1)))
  cnn_single_model.add(MaxPooling1D())
  cnn_single_model.add(Flatten())
  for layer_index in range(num_hidden_layers):
    cnn_single_model.add(Dense(num_hidden_nodes, input_dim=num_logs, activation=activation))
  cnn_single_model.add(Dense(1))
  cnn_single_model.compile(optimizer=optimizer, loss='mse')
  cnn_single_model.fit(X_train, y_train, batch_size=100, epochs=num_epochs, verbose=2)
  
  y_test_predict = []
  for X_test_fragment in X_test:
    X_test_fragment = np.array(X_test_fragment)
    X_test_fragment = X_test_fragment.reshape((1, num_logs, 1))
    y_predict = cnn_single_model.predict(X_test_fragment, verbose=0)
    y_test_predict.append(y_predict[0][0])
  rmse = np.sqrt(mean_squared_error(y_test, y_test_predict))
  y_test_predict = pd.Series(y_test_predict, index=y_test_date)
  return rmse

### CNN multivariate:

> For multiple time series input.



In [ ]:
def cnn_multi_stg1(X_train, y_train, X_validate, y_validate, X_test, y_test, parameters):
  """
  This function takes in inputs and outputs of train, validate and test data as well as parameters to 
  calculate the RMSE between the predictions and reals values from validate and test set.
  :param X_train: input from train set of time series.
  :param y_train: output from train set of time series.
  :param X_validate: input from validate set of time series.
  :param y_validate: output from validate set of time series.
  :param X_test: input from test set of time series.
  :param y_test: output from test set of time series.
  :param parameters: parameters used as the configuration of model.
  :return: Float RMSE value on validate set and RMSE value on test set.
  """
  num_epochs, num_logs, num_hidden_layers, num_hidden_nodes, activation, optimizer = parameters[0], \
  parameters[1], parameters[2], parameters[3], parameters[4], parameters[5]
  
  cnn_multiple_model = keras.Sequential()
  cnn_multiple_model.add(Conv1D(filters=64, kernel_size=2, activation=activation, input_shape=(num_logs*3, 1)))
  cnn_multiple_model.add(MaxPooling1D())
  cnn_multiple_model.add(Flatten())

  for layer_index in range(num_hidden_layers):
    cnn_multiple_model.add(Dense(num_hidden_nodes, input_dim=num_logs*3, activation=activation))
  cnn_multiple_model.add(Dense(1))
  cnn_multiple_model.compile(optimizer=optimizer, loss='mse')
  cnn_multiple_model.fit(X_train, y_train, batch_size=100, epochs=num_epochs, verbose=2)
  
  y_validate_predict = []
  for X_validate_fragment in X_validate:
    X_validate_fragment = np.array(X_validate_fragment)
    X_validate_fragment = X_validate_fragment.reshape((1, num_logs*3, 1))
    y_predict = cnn_multiple_model.predict(X_validate_fragment, verbose=0)
    y_validate_predict.append(y_predict[0][0])
  rmse = np.sqrt(mean_squared_error(y_validate, y_validate_predict))

  y_test_predict = []
  for X_test_fragment in X_test:
    X_test_fragment = np.array(X_test_fragment)
    X_test_fragment = X_test_fragment.reshape((1, num_logs*3, 1))
    y_predict = cnn_multiple_model.predict(X_test_fragment, verbose=0)
    y_test_predict.append(y_predict[0][0])
  test_rmse = np.sqrt(mean_squared_error(y_test, y_test_predict))
  return rmse, test_rmse

In [ ]:
def cnn_multivariate_stg1(datasets, parameters):
  """
  This function takes in all train-validate-test sets for time series as well as parameters to 
  calculate the RMSE between the predictions and reals values from validate and test set.
  :param datasets: list of all train-validate-test sets for time series.
  :param parameters: parameters used as the configuration of model.
  :return: Float RMSE values on all validate sets and RMSE values on all test sets.
  """
  train1, train2, train3, validate1, validate2, validate3, test1, test2, test3 = datasets[0], \
  datasets[1], datasets[2], datasets[3], datasets[4], datasets[5], datasets[6], datasets[7], datasets[8]
  num_logs = parameters[1]
  
  X_train1, y_train1, X_train1_date, y_train1_date = data_num_logs_split(train1, num_logs)
  X_train1 = X_train1.reshape((X_train1.shape[0], X_train1.shape[1], 1))
  X_train2, y_train2, X_train2_date, y_train2_date = data_num_logs_split(train2, num_logs)
  X_train2 = X_train2.reshape((X_train2.shape[0], X_train2.shape[1], 1))
  X_train3, y_train3, X_train3_date, y_train3_date = data_num_logs_split(train3, num_logs)
  X_train3 = X_train3.reshape((X_train3.shape[0], X_train3.shape[1], 1))
  data_combine_X_train = np.concatenate((X_train1, X_train2, X_train3), axis=1)

  X_validate1, y_validate1, X_validate1_date, y_validate1_date = data_num_logs_split(validate1, num_logs)
  X_validate1 = X_validate1.reshape((X_validate1.shape[0], X_validate1.shape[1], 1))
  X_validate2, y_validate2, X_validate2_date, y_validate2_date = data_num_logs_split(validate2, num_logs)
  X_validate2 = X_validate2.reshape((X_validate2.shape[0], X_validate2.shape[1], 1))
  X_validate3, y_validate3, X_validate3_date, y_validate3_date = data_num_logs_split(validate3, num_logs)
  X_validate3 = X_validate3.reshape((X_validate3.shape[0], X_validate3.shape[1], 1))
  data_combine_X_validate = np.concatenate((X_validate1, X_validate2, X_validate3), axis=1)

  X_test1, y_test1, X_test1_date, y_test1_date = data_num_logs_split(test1, num_logs)
  X_test1 = X_test1.reshape((X_test1.shape[0], X_test1.shape[1], 1))
  X_test2, y_test2, X_test2_date, y_test2_date = data_num_logs_split(test2, num_logs)
  X_test2 = X_test2.reshape((X_test2.shape[0], X_test2.shape[1], 1))
  X_test3, y_test3, X_test3_date, y_test3_date = data_num_logs_split(test3, num_logs)
  X_test3 = X_test3.reshape((X_test3.shape[0], X_test3.shape[1], 1))
  data_combine_X_test = np.concatenate((X_test1, X_test2, X_test3), axis=1)

  rmse1, test_rmse1 = cnn_multi_stg1(data_combine_X_train, y_train1, data_combine_X_validate, \
                      y_validate1, data_combine_X_test, y_test1, parameters)
  rmse2, test_rmse2 = cnn_multi_stg1(data_combine_X_train, y_train2, data_combine_X_validate, \
                      y_validate2, data_combine_X_test, y_test2, parameters)
  rmse3, test_rmse3 = cnn_multi_stg1(data_combine_X_train, y_train3, data_combine_X_validate, \
                      y_validate3, data_combine_X_test, y_test3, parameters)
  return rmse1, rmse2, rmse3, test_rmse1, test_rmse2, test_rmse3

In [ ]:
def cnn_multi_stg2(X_train, y_train, X_test, y_test, parameters):
  """
  This function takes in inputs and outputs of train and test data as well as parameters to 
  calculate the RMSE between the predictions and reals values from test set.
  :param X_train: input from train set of time series.
  :param y_train: output from train set of time series.
  :param X_test: input from test set of time series.
  :param y_test: output from test set of time series.
  :param parameters: parameters used as the configuration of model.
  :return: Float RMSE value on test set.
  """
  num_epochs, num_logs, num_hidden_layers, num_hidden_nodes, activation, optimizer = parameters[0], \
  parameters[1], parameters[2], parameters[3], parameters[4], parameters[5]

  cnn_multiple_model = keras.Sequential()
  cnn_multiple_model.add(Conv1D(filters=64, kernel_size=2, activation=activation, input_shape=(num_logs*3, 1)))
  cnn_multiple_model.add(MaxPooling1D())
  cnn_multiple_model.add(Flatten())

  for layer_index in range(num_hidden_layers):
    cnn_multiple_model.add(Dense(num_hidden_nodes, input_dim=num_logs*3, activation=activation))
  cnn_multiple_model.add(Dense(1))
  cnn_multiple_model.compile(optimizer=optimizer, loss='mse')

  cnn_multiple_model.fit(X_train, y_train, batch_size=100, epochs=num_epochs, verbose=2)

  y_test_predict= []
  for X_test_fragment in X_test:
    X_test_fragment = np.array(X_test_fragment)
    X_test_fragment = X_test_fragment.reshape((1, num_logs*3, 1))
    y_predict = cnn_multiple_model.predict(X_test_fragment, verbose=0)
    y_test_predict.append(y_predict[0][0])

  rmse = np.sqrt(mean_squared_error(y_test, y_test_predict))

  return rmse

In [ ]:
def cnn_multivariate_stg2(datasets, parameters):
  """
  This function takes in all train and test sets for time series as well as parameters to 
  calculate the RMSE between the predictions and reals values from validate and test set.
  :param datasets: list of all train and test sets for time series.
  :param parameters: parameters used as the configuration of model.
  :return: Float RMSE values on all test sets.
  """
  train1, train2, train3, test1, test2, test3 = datasets[0], \
  datasets[1], datasets[2], datasets[3], datasets[4], datasets[5]
  num_logs = parameters[1]
  
  X_train1, y_train1, X_train1_date, y_train1_date = data_num_logs_split(train1, num_logs)
  X_train1 = X_train1.reshape((X_train1.shape[0], X_train1.shape[1], 1))
  X_train2, y_train2, X_train2_date, y_train2_date = data_num_logs_split(train2, num_logs)
  X_train2 = X_train2.reshape((X_train2.shape[0], X_train2.shape[1], 1))
  X_train3, y_train3, X_train3_date, y_train3_date = data_num_logs_split(train3, num_logs)
  X_train3 = X_train3.reshape((X_train3.shape[0], X_train3.shape[1], 1))
  data_combine_X_train = np.concatenate((X_train1, X_train2, X_train3), axis=1)

  X_test1, y_test1, _, _ = data_num_logs_split(test1, num_logs)
  X_test1 = X_test1.reshape((X_test1.shape[0], X_test1.shape[1], 1))
  X_test2, y_test2, _, _ = data_num_logs_split(test2, num_logs)
  X_test2 = X_test2.reshape((X_test2.shape[0], X_test2.shape[1], 1))
  X_test3, y_test3, _, _ = data_num_logs_split(test3, num_logs)
  X_test3 = X_test3.reshape((X_test3.shape[0], X_test3.shape[1], 1))
  data_combine_X_test = np.concatenate((X_test1, X_test2, X_test3), axis=1)

  rmse1 = cnn_multi_stg2(data_combine_X_train, y_train1, data_combine_X_test, y_test1, parameters)
  rmse2 = cnn_multi_stg2(data_combine_X_train, y_train2, data_combine_X_test, y_test2, parameters)
  rmse3 = cnn_multi_stg2(data_combine_X_train, y_train3, data_combine_X_test, y_test3, parameters)

  return rmse1, rmse2, rmse3

# Experiments

> Experiment 1: Vary the number of epochs.

> Experiment 2: Vary the number of logs.

> Experiment 3: Vary the number of hidden layers.

> Experiment 4: Vary the number of nodes in hidden layers.

> Experiment 5: Vary the type of activation function.

> Experiment 6: Vary the type of optimizer function.





## Default Variables

In [ ]:
# default parameters
default_epochs = 100
default_num_logs = 3
default_num_hidden_layers = 7
default_num_hidden_nodes = 50
default_activation = 'tanh'
default_optimizer = 'Adam'

default_parameters = [default_epochs, default_num_logs, default_num_hidden_layers, default_num_hidden_nodes, default_activation, default_optimizer]

In [ ]:
get_rmse = itemgetter(8)

In [ ]:
datasets1 = [gdp_train_stg1, inflation_train_stg1, unemployment_train_stg1, 
        gdp_validate_stg1, inflation_validate_stg1, unemployment_validate_stg1, 
        gdp_test_stg1, inflation_test_stg1, unemployment_test_stg1]

datasets2 = [gdp_train_stg2, inflation_train_stg2, unemployment_train_stg2, gdp_test_stg2, inflation_test_stg2, unemployment_test_stg2]

datasets_name = ["GDP", "Inflation", "Unemployment"]

In [ ]:
headers = ["Data", "Model", "Epochs", "Logs", "Layers", "Nodes", "Activitation", "Optimizer", "RMSE"]

In [ ]:
# Experiment 1: Vary the number of epochs
num_epochs_list = [10, 50, 100, 150]

# Experiment 2: Vary the number of logs
num_logs_list = [3, 7, 11, 15] #[3, 4, 5, 6, 7, 8, 9]

# Experiment 3: Vary the number of hidden layers
num_hidden_layers_list = [1, 3, 5, 7, 9]

# Experiment 4: Vary the number of nodes in hidden layers
num_hidden_nodes_list = [10, 20, 50, 100, 200]

# Experiment 5: Vary the type of activation function
activation_list = ['relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'selu', 'tanh', 'elu']

# Experiment 6: Vary the type of optimizer function
optimizer_list = ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']

# Strategy 1

In [ ]:
def expt_result_stg1(expt_num, expt_list, datasets, datasets_name, parameters=default_parameters):
  """
  This function takes in settings of experiment and return the result list
  :param expt_num: integer of which experiment is taken.
  :param expt_list: list contains how to vary the parameter of this experiment.
  :param datasets: train-validate-test sets for all time series.
  :param datasets_name: name of all time series.
  :param parameters: parameters used as the configuration of model.
  :return: list contains results of this experiment.
  """
  expt = []
  arima_parameter = [np.nan] * int(len(parameters))
  train_validate = datasets[:len(datasets_name)*2]
  train_test = datasets[:len(datasets_name)] + datasets[-len(datasets_name):]
  for i in range(int(len(datasets_name))):
    train, validate, test = datasets[i], datasets[i + int(len(datasets_name))], datasets[i + int(len(datasets_name))*2]
    if expt_num != 2:
      arima_parameter[1] = parameters[1]
      arima_rmse = arima(train, validate, arima_parameter)
      test_arima_rmse = arima(train, test, arima_parameter)
      expt.append([datasets_name[i], 'ARIMA'] + arima_parameter + [arima_rmse, test_arima_rmse])
      update_parameters = parameters.copy()
      for item in expt_list: 
        update_parameters[expt_num - 1] = item
        cnn_univariate_rmse, test_cnn_univariate_rmse = cnn_univariate_stg1(train, validate, test, update_parameters)
        expt.append([datasets_name[i], 'CNN univariate'] + update_parameters + [cnn_univariate_rmse, test_cnn_univariate_rmse])
    else:
      update_parameters = parameters.copy()
      for item in expt_list:
        update_parameters[expt_num - 1] = item
        arima_parameter[1] = item
        arima_rmse = arima(train, validate, arima_parameter)
        test_arima_rmse = arima(train, test, arima_parameter)
        expt.append([datasets_name[i], 'ARIMA'] + arima_parameter + [arima_rmse, test_arima_rmse])

        cnn_univariate_rmse, test_cnn_univariate_rmse = cnn_univariate_stg1(train, validate, test, update_parameters)
        expt.append([datasets_name[i], 'CNN univariate'] + update_parameters + [cnn_univariate_rmse, test_cnn_univariate_rmse])        
  for item in expt_list:
    update_parameters[expt_num - 1] = item      
    cnn_multivariate_rmse1, cnn_multivariate_rmse2, cnn_multivariate_rmse3, test_cnn_multivariate_rmse1, \
    test_cnn_multivariate_rmse2, test_cnn_multivariate_rmse3 = cnn_multivariate_stg1(datasets, update_parameters)
    cnn_multivariate_rmse = [cnn_multivariate_rmse1, cnn_multivariate_rmse2, cnn_multivariate_rmse3]
    test_cnn_multivariate_rmse = [test_cnn_multivariate_rmse1, test_cnn_multivariate_rmse2, test_cnn_multivariate_rmse3]
    for i in range(int(len(datasets_name))):
      expt.append([datasets_name[i], 'CNN multivariate'] + update_parameters + [cnn_multivariate_rmse[i], test_cnn_multivariate_rmse[i]])
  return expt

In [ ]:
def expt_sorted_models_stg1(expt):
  """
  This function takes in a list contains the result of an experiment.
  :param expt: list contains the result of an experiment.
  :return: sorted list of all models.
  """
  rmse_dict = {}
  for i in range(len(expt)):
    dict_key = str(expt[i][1:-2])
    if dict_key in rmse_dict.keys():
      rmse_dict[dict_key].append([expt[i][8], expt[i][9]])
    else:
      rmse_dict[dict_key] = [[expt[i][8], expt[i][9]]]  
  average_rmse, average_test_rmse = {}, {}
  name_of_models = {}
  count = 0
  for key in rmse_dict:
    model_rmse, model_test_rmse = [], []
    for j in range(len(rmse_dict[key])):
      model_rmse.append(rmse_dict[key][j][0])
      model_test_rmse.append(rmse_dict[key][j][1])
    average_rmse[count] = np.mean(model_rmse)
    average_test_rmse[count] = np.mean(model_test_rmse)
    name_of_models[count] = key
    count += 1 
  sorted_models = []
  for key in name_of_models:
    sorted_models.append([key, average_rmse[key], average_test_rmse[key], name_of_models[key]])
  final_sorted_models = sorted(sorted_models, key=itemgetter(1)) 
  return final_sorted_models

In [ ]:
def expt_best_print(sorted_models, best_num, summary=False):
  """
  This function takes in a sorted list, the number to print and whether
  the sorted list is the summary list to print the best_num best models.
  :param sorted_models: sorted list contains mean RMSE for each model.
  :param best_num: the number of model to print.
  :param summary: whether the sorted list is the summary list.
  :return:
  """
  for algo in ["ARIMA", "CNN univariate", "CNN multivariate"]:
    count = 0
    best_model_index = []
    for i in range(len(sorted_models)):
      model_parameter = sorted_models[i][3][2:-2].replace("'", '').split(',')
      if summary:
        if model_parameter[0] == algo and (model_parameter[1] == " nan" or int(model_parameter[1]) == 100):
          best_model_index.append(i)
          count += 1
          if count == best_num:
            break
      else:
        if model_parameter[0] == algo:
          best_model_index.append(i)
          count += 1
          if count == best_num:
            break
    for index in best_model_index:
      print("The best {} {}: {}, RMSE = {}, Test RMSE = {}".format(algo, index+1, sorted_models[index][3], \
                sorted_models[index][1], sorted_models[index][2]))
  return

# Experiments using the first strategy

In [ ]:
expt1_stg1 = expt_result_stg1(1, num_epochs_list, datasets1, datasets_name, parameters=default_parameters)
expt2_stg1 = expt_result_stg1(2, num_logs_list, datasets1, datasets_name, parameters=default_parameters)
expt3_stg1 = expt_result_stg1(3, num_hidden_layers_list, datasets1, datasets_name, parameters=default_parameters)
expt4_stg1 = expt_result_stg1(4, num_hidden_nodes_list, datasets1, datasets_name, parameters=default_parameters)
expt5_stg1 = expt_result_stg1(5, activation_list, datasets1, datasets_name, parameters=default_parameters)
expt6_stg1 = expt_result_stg1(6, optimizer_list, datasets1, datasets_name, parameters=default_parameters)

In [ ]:
expt1_sorted_models_stg1 = expt_sorted_models_stg1(expt1_stg1)
expt2_sorted_models_stg1 = expt_sorted_models_stg1(expt2_stg1)
expt3_sorted_models_stg1 = expt_sorted_models_stg1(expt3_stg1)
expt4_sorted_models_stg1 = expt_sorted_models_stg1(expt4_stg1)
expt5_sorted_models_stg1 = expt_sorted_models_stg1(expt5_stg1)
expt6_sorted_models_stg1 = expt_sorted_models_stg1(expt6_stg1)

stg1 = expt1_stg1 + expt2_stg1 + expt3_stg1 + expt4_stg1 + expt5_stg1 + expt6_stg1
stg1_sorted_models = expt_sorted_models_stg1(stg1)

In [ ]:
print("Experiment 1:")
expt_best_print(expt1_sorted_models_stg1, 1)
print("\n\nExperiment 2:")
expt_best_print(expt2_sorted_models_stg1, 1)
print("\n\nExperiment 3:")
expt_best_print(expt3_sorted_models_stg1, 1)
print("\n\nExperiment 4:")
expt_best_print(expt4_sorted_models_stg1, 1)
print("\n\nExperiment 5:")
expt_best_print(expt5_sorted_models_stg1, 1)
print("\n\nExperiment 6:")
expt_best_print(expt6_sorted_models_stg1, 1)
print("\n\nExperiment Summary:")
expt_best_print(stg1_sorted_models, 1, summary=True)

# Strategy 2

In [ ]:
def expt_result_stg2(expt_num, expt_list, datasets, datasets_name, parameters=default_parameters):
  """
  This function takes in settings of experiment and return the result list
  :param expt_num: integer of which experiment is taken.
  :param expt_list: list contains how to vary the parameter of this experiment.
  :param datasets: train and test sets for all time series.
  :param datasets_name: name of all time series.
  :param parameters: parameters used as the configuration of model.
  :return: list contains results of this experiment.
  """
  expt = []
  arima_parameter = [np.nan] * int(len(parameters))

  for i in range( int( len(datasets_name) ) ):
    train, test = datasets[i], datasets[i + int(len(datasets_name))]

    if expt_num != 2:
      arima_parameter[1] = parameters[1]
      arima_rmse = arima(train, test, arima_parameter)
      expt.append([datasets_name[i], 'ARIMA'] + arima_parameter + [arima_rmse])

      update_parameters = parameters.copy()
      for item in expt_list: 
        update_parameters[expt_num - 1] = item
        
        cnn_univariate_rmse = cnn_univariate_stg2(train, test, update_parameters)
        expt.append([datasets_name[i], 'CNN univariate'] + update_parameters + [cnn_univariate_rmse])
    else:
      update_parameters = parameters.copy()
      for item in expt_list:
        update_parameters[expt_num - 1] = item

        arima_parameter[1] = item
        arima_rmse = arima(train, test, arima_parameter)
        expt.append([datasets_name[i], 'ARIMA'] + arima_parameter + [arima_rmse])

        cnn_univariate_rmse = cnn_univariate_stg2(train, test, update_parameters)
        expt.append([datasets_name[i], 'CNN univariate'] + update_parameters + [cnn_univariate_rmse])
  
  for item in expt_list:
    update_parameters[expt_num - 1] = item      
    cnn_multivariate_rmse1, cnn_multivariate_rmse2, cnn_multivariate_rmse3 = \
    cnn_multivariate_stg2(datasets, update_parameters)
    cnn_multivariate_rmse = [cnn_multivariate_rmse1, cnn_multivariate_rmse2, cnn_multivariate_rmse3]
    for i in range(int(len(datasets_name))):
      expt.append([datasets_name[i], 'CNN multivariate'] + update_parameters + [cnn_multivariate_rmse[i]])

  return expt

In [ ]:
def expt_sorted_models_stg2(expt):
  """
  This function takes in a list contains the result of an experiment.
  :param expt: list contains the result of an experiment.
  :return: list contains rmse value for each model and a sorted list of all models.
  """
  rmse_dict = {}
  for i in range(len(expt)):
    dict_key = str(expt[i][1:-1])
    if dict_key in rmse_dict.keys():
      if expt[i][8] not in rmse_dict[dict_key]:
        rmse_dict[dict_key].append(expt[i][8])
    else:
      rmse_dict[dict_key] = [expt[i][8]]  
  models_rmse = [] 
  name_of_models = {}
  count = 0
  for key in rmse_dict:
    models_rmse.append(rmse_dict[key])
    name_of_models[count] = key
    count += 1  
  rmse_values = [*models_rmse]
  average_rmse = [np.mean(x) for x in rmse_values]  
  sorted_models = []
  for key in name_of_models:
    sorted_models.append([key,average_rmse[key], name_of_models[key]])
  get_rmse = itemgetter(1)
  final_sorted_models = sorted(sorted_models, key=get_rmse)   
  return models_rmse, final_sorted_models

In [ ]:
def expt_datasets(expt, expt_num):
  """
  This function takes in a list contains the result of an experiment and the experiment number
  to return experiment number, how the variable varying, the mean RMSE for each algorithm.
  :param expt: list contains the result of an experiment.
  :param expt_num: the number of experiment.
  :return: experiment number, lists contains how the variable varying, lists contains the mean RMSE for each algorithm.
  """
  labels, arima_rmse, cnn_univariate_rmse, cnn_multivariate_rmse = [], {}, {}, {}
  for i in range(len(expt)):
    if expt[i][1] != "ARIMA" and expt[i][expt_num + 1] not in labels:
      labels.append(expt[i][expt_num + 1])
    if expt[i][1] == 'ARIMA':
      if expt[i][expt_num + 1] in arima_rmse.keys():
        arima_rmse[expt[i][expt_num + 1]].append(expt[i][8])
      else:
        arima_rmse[expt[i][expt_num + 1]] = [expt[i][8]]
    elif expt[i][1] == 'CNN univariate':
      if expt[i][expt_num + 1] in cnn_univariate_rmse.keys():
        cnn_univariate_rmse[expt[i][expt_num + 1]].append(expt[i][8])
      else:
        cnn_univariate_rmse[expt[i][expt_num + 1]] = [expt[i][8]]
    else:
      if expt[i][expt_num + 1] in cnn_multivariate_rmse.keys():
        cnn_multivariate_rmse[expt[i][expt_num + 1]].append(expt[i][8])
      else:
        cnn_multivariate_rmse[expt[i][expt_num + 1]] = [expt[i][8]]
  for rmse_dict in [arima_rmse, cnn_univariate_rmse, cnn_multivariate_rmse]:
    for k in rmse_dict.keys():
      v = list(rmse_dict[k])
      mean_rmse = sum(v) / len(v)
      rmse_dict[k] = mean_rmse 
  return expt_num, labels, arima_rmse, cnn_univariate_rmse, cnn_multivariate_rmse

In [ ]:
def expt_plot(lists):
  """
  This function takes in results of the expt_datasets(expt, expt_num) function and plot them.
  :param lists: results of the expt_datasets(expt, expt_num) function.
  :return:
  """  
  expt_num, labels, arima_rmse, cnn_univariate_rmse, cnn_multivariate_rmse = lists[0], \
  lists[1], lists[2], lists[3], lists[4]
  expt_name = headers[2:-1]
  arima_rmse = list(arima_rmse.values()) * len(cnn_univariate_rmse) \
  if len(arima_rmse) == 1 else list(arima_rmse.values())
  cnn_univariate_rmse, cnn_multivariate_rmse = list(cnn_univariate_rmse.values()), \
  list(cnn_multivariate_rmse.values())

  plt.figure(figsize=(8,4))
  plt.plot(arima_rmse, color='red', label='ARIMA')
  plt.plot(cnn_univariate_rmse, color='green', label='CNN univariate')
  plt.plot(cnn_multivariate_rmse, color='blue', label='CNN multivariate')
  plt.legend()
  plt.xticks(ticks=np.arange(len(labels)), labels=labels)
  plt.grid()
  plt.xlabel(expt_name[expt_num-1])
  plt.ylabel("RMSE")
  plt.title("Varying "+expt_name[expt_num-1])

In [ ]:
def expt_friedman_conover(expt, expt_num):
  """
  This function takes in a list contains the result of an experiment and the number of experiment
  to print the result of Friedman test and Conover test if the p-value of Friedman test is less
  than 0.05.
  :param expt: list contains the result of an experiment.
  :param expt_num: the number of experiment.
  :return:
  """ 
  dataset_friedman, final_sorted_models = expt_sorted_models_stg2(expt)
  for i in range(len(dataset_friedman)):
    if len(dataset_friedman[i]) != 3:
      value = np.mean(dataset_friedman[i])
      dataset_friedman[i] = [value] * 3
  rmse_values = [*dataset_friedman]
  transpose_rmse_values = np.array(rmse_values).T.tolist()

  print("Printing out the complete data passed to friedman test: ", dataset_friedman)
  stat, p = friedmanchisquare(*dataset_friedman)
  print("Friedman test : ")
  print('p-value = %.5f' % (p))
  alpha = 0.05
  if p > alpha:
    print('Fail to reject H0.')
  else:
    print('Reject H0.')

    # The post hoc Conover test.
    average_rmse = [np.mean(x) for x in rmse_values]
    conover = posthoc_conover_friedman(transpose_rmse_values)
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    print("conover :\n", conover)

    # Locate the the significant difference.
    for i in range(conover.shape[0]):
      for j in range(i+1, conover.shape[1]):
        if conover.iloc[i,j] < alpha:
          print("\n | CONOVER |Found indication of difference (p = ", conover.iloc[i,j],") between network ", i, " and ",j)

    print("\nSorted models in order of RMSE : \n")
    for model in final_sorted_models:
      print("Model {0} : mean RMSE = {1:.5f} for {2}".format(*model))
  return

# Expermiments using the second strategy

In [ ]:
expt1_stg2 = expt_result_stg2(1, num_epochs_list, datasets2, datasets_name, parameters=default_parameters)
expt2_stg2 = expt_result_stg2(2, num_logs_list, datasets2, datasets_name, parameters=default_parameters)
expt3_stg2 = expt_result_stg2(3, num_hidden_layers_list, datasets2, datasets_name, parameters=default_parameters)
expt4_stg2 = expt_result_stg2(4, num_hidden_nodes_list, datasets2, datasets_name, parameters=default_parameters)
expt5_stg2 = expt_result_stg2(5, activation_list, datasets2, datasets_name, parameters=default_parameters)
expt6_stg2 = expt_result_stg2(6, optimizer_list, datasets2, datasets_name, parameters=default_parameters)

In [ ]:
expt_plot(expt_datasets(expt1_stg2, 1))
expt_friedman_conover(expt1_stg2, 1)

In [ ]:
expt_plot(expt_datasets(expt2_stg2, 2))
expt_friedman_conover(expt2_stg2, 2)

In [ ]:
expt_plot(expt_datasets(expt3_stg2, 3))
expt_friedman_conover(expt3_stg2, 3)

In [ ]:
expt_plot(expt_datasets(expt4_stg2, 4))
expt_friedman_conover(expt4_stg2, 4)

In [ ]:
expt_plot(expt_datasets(expt5_stg2, 5))
expt_friedman_conover(expt5_stg2, 5)

In [ ]:
expt_plot(expt_datasets(expt6_stg2, 6))
expt_friedman_conover(expt6_stg2, 6)

# Grid Search

In [ ]:
def grid_stg2_sorted_models_epoch_select(sorted_rmse, sorted_models, length):
  """
  This function takes in a sorted rmse list, a sorted model list, the number of
  epochs and select length best models return a list contains length best rmse 
  and a list contains length best models.
  :param sorted_rmse: sorted list contains RMSE for each model as a list.
  :param sorted_models: sorted list contains mean RMSE for each model.
  :param length: the number of models to select.
  :return: a list contains length best rmse and a list contains length best models.
  """
  stg2_rmse_select = []
  stg2_models_select = []
  count = 0
  for i in range(len(sorted_models)):
    algo = sorted_models[i][2][2:-2].replace("'", '').split(',')
    if algo[0] == "CNN univariate" or algo[0] == "CNN multivariate":
      stg2_rmse_select.append(sorted_rmse[i])
      stg2_models_select.append(sorted_models[i])
      count += 1
    if count == length:
      break
  return stg2_rmse_select, stg2_models_select

In [ ]:
grid_stg2 = []
arima_parameters, cnn_parameters = [], []
arima_empty_parameter = [np.nan] * 6
for log in [3,7]:
  arima_parameter = arima_empty_parameter.copy()
  arima_parameter[1] = log
  arima_parameters.append(arima_parameter)
  for layer in [1,3]:
    for node in [50,100]:
      for activation in ['relu','tanh']:
        for optimizer in ['SGD','Adam']:
          cnn_parameters.append([100, log, layer, node, activation, optimizer])

In [ ]:
for i in range(int(len(datasets_name))):
  train, test = datasets2[i], datasets2[i + int(len(datasets_name))]
    
  for arima_parameter in arima_parameters:
    arima_rmse = arima(train, test, arima_parameter)
    grid_stg2.append([datasets_name[i], 'ARIMA'] + arima_parameter + [arima_rmse])

  for cnn_parameter in cnn_parameters:
    cnn_univariate_rmse = cnn_univariate_stg2(train, test, cnn_parameter)
    grid_stg2.append([datasets_name[i], 'CNN univariate'] + cnn_parameter + [cnn_univariate_rmse])

for cnn_parameter in cnn_parameters:           
  cnn_multivariate_rmse1, cnn_multivariate_rmse2, cnn_multivariate_rmse3 = \
  cnn_multivariate_stg2(datasets2, cnn_parameter)
  cnn_multivariate_rmse = [cnn_multivariate_rmse1, cnn_multivariate_rmse2, cnn_multivariate_rmse3]
  for i in range(int(len(datasets_name))):
    grid_stg2.append([datasets_name[i], 'CNN multivariate'] + cnn_parameter + [cnn_multivariate_rmse[i]])

In [ ]:
grid_stg2_sorted_rmse, grid_stg2_sorted_models = expt_sorted_models_stg2(grid_stg2)
grid_best_10_cnn_rmse, grid_best_10_cnn_models = \
grid_stg2_sorted_models_epoch_select(grid_stg2_sorted_rmse, grid_stg2_sorted_models, 10)
print("\nSorted models in order of RMSE : \n")
for model in grid_best_10_cnn_models:
  print("Model {0} : mean RMSE = {1:.5f} for {2}".format(*model))

In [ ]:
for i in range(len(grid_best_10_cnn_rmse)):
  if len(grid_best_10_cnn_rmse[i]) != 3:
    value = np.mean(grid_best_10_cnn_rmse[i])
    grid_best_10_cnn_rmse[i] = [value] * 3
grid_rmse_values = [*grid_best_10_cnn_rmse]

# compare samples
print("Printing out the complete data passed to friedman test: ", grid_best_10_cnn_rmse)
stat, p = friedmanchisquare(*grid_best_10_cnn_rmse)
print("Friedman test : ")
print('p-value = %.5f' % (p))
# interpret
alpha = 0.05
if p > alpha:
  print('Fail to reject H0.')
else:
  print('Reject H0.')

In [ ]:
for i in range(len(grid_stg2_sorted_models)):
  algo = grid_stg2_sorted_models[i][2][2:-2].replace("'", '').split(',')
  if algo[0] == "ARIMA":
    grid_best_arima_index = i
    break
grid_best_arima_rmse = grid_stg2_sorted_rmse[grid_best_arima_index]
print('The best ARIMA model is:\n', grid_stg2_sorted_models[grid_best_arima_index])

In [ ]:
grid_best_10_cnn_rmse_with_arima = grid_best_10_cnn_rmse.copy()
grid_best_10_cnn_rmse_with_arima.append(grid_best_arima_rmse)
for i in range(len(grid_best_10_cnn_rmse_with_arima)):
  if len(grid_best_10_cnn_rmse_with_arima[i]) != 3:
    value = np.mean(grid_best_10_cnn_rmse_with_arima[i])
    grid_best_10_cnn_rmse_with_arima[i] = [value] * 3
grid_rmse_values = [*grid_best_10_cnn_rmse_with_arima]

# compare samples
print("Printing out the complete data passed to friedman test: ", grid_rmse_values)
stat, p = friedmanchisquare(*grid_rmse_values)
print("Friedman test : ")
print('p-value = %.5f' % (p))
# interpret
alpha = 0.05
if p > alpha:
  print('Fail to reject H0.')
else:
  print('Reject H0.')